In [1]:
!pip install 'zarr<3'

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")

✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p


## Load Dataset

In [2]:
# Cell 3: Import and create dataloaders
from crc_dataset import Config, create_dataloaders

config = Config("config.yaml")
config.print_config()

train_loader, val_loader, test_loader = create_dataloaders(config)

print("\n✅ Ready to use!")

✅ CRC Dataset Package v1.0.0 loaded
CRC DATASET CONFIGURATION
Config File: config.yaml
Zarr Base: /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/ORION-CRC/zarr_data
Cache: ./cache/crc_dataset_cache.pkl
Strategy: memory

📊 Data Splits:
  Train: 28 samples
  Val: 6 samples
  Test: 7 samples

🔄 DataLoader:
  Batch Size: 4
  Num Workers: 4
  Pin Memory: True

🎨 Augmentation:
  Training: True
  Probability: 0.5

🎯 HV Maps:
  Generate: True
  Method: pannuke
  HE Nuclei: True
  HE Cells: True
  MIF Nuclei: True
  MIF Cells: True

🔍 Filtering:
  Min Instances: 0
  Filter Empty: True

CREATING DATALOADERS
Strategy: memory
Use Cache: True
Batch Size: 4
Num Workers: 4

📦 Loading from cache: ./cache/crc_cache_train_82d4f49c.pkl
📦 Loaded 1987 patches from cache
📦 Loading from cache: ./cache/crc_cache_val_79e4cd4c.pkl
📦 Loaded 944 patches from cache
📦 Loading from cache: ./cache/crc_cache_val_81ebba58.pkl
📦 Loaded 406 patches from cache
✅ Dataloaders created:
   Train: 1987 patches (496 b

## Model

In [3]:
# In your notebook.ipynb
from models import DualEncoderUNet, train_model, set_seed

# Create model
model = DualEncoderUNet(backbone='resnet101', pretrained=True, dropout_rate=0.3)

# Print model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")



/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total parameters: 238,139,724
Trainable parameters: 238,139,724


## Tranier

In [ ]:
# Train model (loader must provide 'he_image' and 'mif_image' separately now)
trained_model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=50,
    lr=1e-4,
    weight_decay=1e-4
)

Epoch 1/50 [Train]:  80%|███████▉  | 395/496 [01:13<00:16,  6.10it/s, loss=0.5034]

## metrics

In [5]:
import torch
from metrics import (
    dice_coefficient, 
    iou_score, 
    panoptic_quality,
    aggregated_jaccard_index,
    compute_batch_metrics,
    print_metrics
)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move model to device
model = model.to(device)
print(f"✅ Model moved to {device}")

# During evaluation
model.eval()
with torch.no_grad():
    for batch in val_loader:
        he_img = batch['he_image'].to(device)
        mif_img = batch['mif_image'].to(device)
        
        # Forward pass
        outputs = model(he_img, mif_img)
        
        # Compute metrics for this batch
        batch_metrics = compute_batch_metrics(outputs, batch, device)
        
        # Print
        print_metrics(batch_metrics, title="Validation Metrics")
        
        # Only run one batch for testing
        break

# Example: Manual metric computation
# Example: Manual metric computation
pred_mask = outputs['he_nuclei_seg'][0].cpu()  # Move to CPU
gt_mask = batch['he_nuclei_mask'][0].unsqueeze(0).cpu()  # Move to CPU

dice = dice_coefficient(pred_mask, gt_mask)
iou = iou_score(pred_mask, gt_mask)

print(f"\nManual computation:")
print(f"Dice: {dice:.4f}, IoU: {iou:.4f}")

Using device: cuda
✅ Model moved to cuda

Validation Metrics

HE NUCLEI:
  dice        : 0.3110
  iou         : 0.1853
  pq          : 0.0000
  aji         : 0.0000

HE CELL:
  dice        : 0.5413
  iou         : 0.3723
  pq          : 0.0000
  aji         : 0.0003

MIF NUCLEI:
  dice        : 0.4257
  iou         : 0.2728
  pq          : 0.0000
  aji         : 0.0000

MIF CELL:
  dice        : 0.5156
  iou         : 0.3484
  pq          : 0.0000
  aji         : 0.0000


Manual computation:
Dice: 0.3361, IoU: 0.2020
